In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1' # nvidia-smi로 비어있는 gpu 확인하고 여기서 선택할것!

In [2]:
import transformers
from evaluate import load
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

import json

import torch
from torch.utils.data import Dataset, DataLoader

import numpy as np

from tqdm import tqdm

import re

from datasets import load_dataset


ImportError: cannot import name 'DataCollatorForSeq2Seq' from 'transformers' (/opt/anaconda3/lib/python3.7/site-packages/transformers/__init__.py)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
METRIC_NAME = "bleu"
MODEL_CHECKPOINT = "facebook/bart-large"
MAX_INPUT_LENGTH = 1024
MAX_TARGET_LENGTH = 256

In [ ]:
metric = load(METRIC_NAME)


tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)


In [ ]:
raw_datasets = load_dataset("daily_dialog")

In [ ]:
class bf_Dataset(Dataset):
    def __init__(self, raw_datasets = raw_datasets, train_val_test=0, tokenizer = None):
        if train_val_test == 0:
            self.data = raw_datasets['train']["dialog"]
        elif train_val_test == 1:
            self.data = raw_datasets['validation']["dialog"]
            
        else:
            self.data = raw_datasets['test']["dialog"]
        
        self.tokenizer = tokenizer
        self.inputs = []
        self.outputs = []
        for dg in self.data:
            for turns in range(len(dg)//2,len(dg)):
                input_string = ""
                for input_turns in dg[:turns]:
                    input_string += input_turns.strip() + " "
                self.inputs.append(input_string)
                self.outputs.append(dg[turns].strip())
        
    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        embeddings = self.tokenizer(self.inputs[idx], max_length=MAX_INPUT_LENGTH, truncation=True)
        embeddings['labels'] = self.tokenizer(self.outputs[idx], max_length=MAX_TARGET_LENGTH, truncation=True, return_attention_mask=False)["input_ids"]
        return embeddings

class test_Dataset(Dataset):
    def __init__(self, raw_datasets = raw_datasets, train_val_test=0, tokenizer = None):
        self.data = raw_datasets['test']["dialog"]
        
        self.tokenizer = tokenizer
        self.inputs = []
        self.outputs = []
        for dg in self.data:
            for turns in range(len(dg)//2,len(dg)):
                input_string = ""
                for input_turns in dg[:turns]:
                    input_string += input_turns.strip() + " "
                self.inputs.append(input_string)
                self.outputs.append(dg[turns].strip())
        
    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        embeddings = self.tokenizer(self.inputs[idx], max_length=MAX_INPUT_LENGTH, truncation=True, return_tensors="pt")
        embeddings['labels'] = self.tokenizer(self.outputs[idx], max_length=MAX_TARGET_LENGTH, truncation=True, return_attention_mask=False,return_tensors="pt")["input_ids"]
        return embeddings

In [ ]:
train_ds = bf_Dataset(raw_datasets=raw_datasets, train_val_test=0, tokenizer=tokenizer)
val_ds = bf_Dataset(raw_datasets=raw_datasets, train_val_test=1, tokenizer=tokenizer)
test_ds = test_Dataset(raw_datasets=raw_datasets, train_val_test=2, tokenizer=tokenizer)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)

In [3]:
model_name = MODEL_CHECKPOINT.split("/")[-1]

NameError: name 'MODEL_CHECKPOINT' is not defined

In [4]:
batch_size = 8
epochs = 4


args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-dailydialogue-vanila",
    evaluation_strategy = "steps",
    save_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    load_best_model_at_end=True,
    save_steps=3000,
    eval_steps=3000,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=epochs,
    predict_with_generate=True,
    fp16=True,
)

NameError: name 'Seq2SeqTrainingArguments' is not defined

In [11]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [12]:

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    

    
    rt = {"bleu":0}
    for i,v in enumerate(decoded_labels): 
        result = metric.compute(predictions=[decoded_preds[i]], references=[[v]], max_order = 2)
        rt["bleu"] += result["bleu"]
    rt["bleu"] = rt["bleu"] / len(decoded_labels) * 100
    return rt

In [13]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [14]:
trainer.train()

/home/leadawon5/dawon/myvenv/lib/python3.7/site-packages/transformers/optimization.py:415: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Bleu
3000,2.337700,2.212464,4.782108
6000,2.116300,2.274352,6.057237
9000,1.985100,2.093913,5.737030
12000,1.775000,2.070626,6.838622
15000,1.756400,2.054525,7.182127
18000,1.596300,2.042914,7.578293
21000,1.594700,2.033999,7.627586


TrainOutput(global_step=22768, training_loss=1.9280099111464521, metrics={'train_runtime': 7330.1423, 'train_samples_per_second': 24.847, 'train_steps_per_second': 3.106, 'total_flos': 9.413602510009958e+16, 'train_loss': 1.9280099111464521, 'epoch': 4.0})

In [15]:
assert False

AssertionError: 

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(f"{model_name}-finetuned-dailydialogue-vanila/checkpoint-21000")

model.eval()


In [ ]:
filename = "./output_dd.txt"
with open(filename,"w") as f:
    pass

pp = 0

with torch.no_grad():
    # suma = {'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}
    model.to(device)
    for i,embeddings in enumerate(test_ds):
        output = model.generate(embeddings["input_ids"].to(device))[0]
        
        pred = tokenizer.decode(output.cpu(), skip_special_tokens=True)
        
        gt = tokenizer.decode(embeddings["labels"][0], skip_special_tokens=True)
        stringt = "input_ids"
        with open(filename,"a") as f:
            f.write(f"{i} 번째 문장\ninput_ids : \n{tokenizer.decode(embeddings[stringt][0],skip_special_tokens=True)}\nprediction: \n{pred}\ngt        : \n{gt}\n\n")

        pp += metric.compute(predictions=[pred], references=[[gt]], max_order = 2)["bleu"]
    pp = pp / len(test_ds) * 100
print("Bleu 2-gram : ",pp)

        

In [ ]:
print(pp)